##### Padronizacao

In [1]:
import pandas as pd

dadosDiadema = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\diadema,-são paulo-air-quality.csv")
dadosItaim = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\itaim-paulista, são paulo-air-quality.csv")
dadosItaquera = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\itaquera,-são paulo-air-quality.csv")
dadosMooca = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\mooca,-são paulo-air-quality.csv")
dadosSantoandre = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\s.andré-capuava,-são paulo-air-quality.csv")
dadosSaoBernardo = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\s.bernardo-centro,-são paulo-air-quality.csv")

In [4]:
import pandas as pd
import glob

# Dicionário com os dados de latitude e longitude por região
coordenadas = {
    "Sao Bernardo": {"latitude": -23.701934820154197, "longitude": -46.553310586836886},
    "Santo Andre": {"latitude": -23.658138559401298, "longitude": -46.48984790027418},
    "Diadema": {"latitude": -23.685313382167138, "longitude": -46.625954889892164},
    "Itaquera": {"latitude": -23.541638970943424, "longitude": -46.47125736949073},
    "Itaim Paulista": {"latitude": -23.499440086125986, "longitude": -46.39547580760927},
    "Mooca": {"latitude": -23.55817427309982, "longitude": -46.60807618968641},
}

# Caminho para todos os arquivos CSV
arquivos_csv = glob.glob("*.csv")  # Ajuste o caminho se necessário

# Processa cada arquivo CSV
for arquivo in arquivos_csv:
    # Lê o arquivo
    df = pd.read_csv(arquivo)
    
    # Extrai a região do nome do arquivo (assumindo que está no nome, ex.: "Santo Andre.csv")
    regiao = arquivo.split(".csv")[0]
    
    # Adiciona latitude e longitude, se a região existir no dicionário
    if regiao in coordenadas:
        df["latitude"] = coordenadas[regiao]["latitude"]
        df["longitude"] = coordenadas[regiao]["longitude"]
    else:
        print(f"Região não encontrada no dicionário: {regiao}")
        continue

    # Salva o arquivo atualizado
    df.to_csv(f"{regiao}_atualizado.csv", index=False)

print("Processamento concluído.")


Processamento concluído.


In [ ]:
import pymysql
import csv
from datetime import datetime
import glob

# Conexão com o banco de dados
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='525748',
    database='bd_medicao'
)

# Função para converter valores em string para float, tratando espaços em branco ou valores inválidos
def convert_to_float(value):
    try:
        return float(value.strip()) if value.strip() else 0.0  # Substituir vazio por 0.0
    except ValueError:
        return 0.0  # Substituir erro de conversão por 0.0

# Função para processar um arquivo CSV
def process_csv_file(csv_file_path, connection):
    with open(csv_file_path, encoding='utf-8') as csv_file:
        csv_data = csv.reader(csv_file)
        next(csv_data)  # Ignora o cabeçalho

        cursor = connection.cursor()
        for row in csv_data:
            try:
                # Ajustar o formato da data para 'YYYY/MM/DD'
                date = datetime.strptime(row[0].strip(), '%Y/%m/%d')

                # Extrair os valores dos poluentes com a função convert_to_float
                pm25 = convert_to_float(row[1])
                pm10 = convert_to_float(row[2])
                o3 = convert_to_float(row[3])
                no2 = convert_to_float(row[4])
                so2 = convert_to_float(row[5]) if len(row) > 5 else 0.0
                co = convert_to_float(row[6]) if len(row) > 6 else 0.0

                # Inserir no banco de dados
                cursor.execute("""
                    INSERT INTO dados_poluentes (tempo_registro, pm25, pm10, o3, no2, so2, co) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                """, (date, pm25, pm10, o3, no2, so2, co))

            except Exception as e:
                print(f"Erro ao processar linha: {row}. Erro: {e}")

        connection.commit()
        print(f"Arquivo {csv_file_path} processado com sucesso!")

# Caminho para os arquivos CSV
arquivos_csv = glob.glob("caminho_para_seus_csvs/*.csv")  # Ajuste o caminho aqui

# Processa cada arquivo CSV
for arquivo in arquivos_csv:
    process_csv_file(arquivo, connection)

# Fecha a conexão com o banco
connection.close()
print("Todos os arquivos foram processados e os dados inseridos no banco!")